# Automate Event Planning

- Import libraries, APIs and LLM

In [ ]:
from crewai import Agent, Crew, Task

In [ ]:
import os
import getpass

# Prompt the user for input (hidden)
api_key = getpass.getpass("Enter the openai api key to set it as environment variable 'OPENAI_API_KEY': ")
api_base = getpass.getpass("Enter the openai api base to set it as environment variable 'OPENAI_API_BASE': ")
# serper_api_key = getpass.getpass("Enter the SERPER_API_KEY for efficient search: ")
os.environ["SERPER_API_KEY"] = getpass.getpass("Enter the serper api key to set it as environment variable 'SERPER_API_KEY': ")

os.environ["OPENAI_API_KEY"] = api_key
os.environ["OPENAI_API_BASE"] = api_base
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## crewAI Tools

In [3]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

# Initialize the tools
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

/opt/homebrew/lib/python3.10/site-packages/pydantic/_internal/_config.py:295: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
/opt/homebrew/lib/python3.10/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "schema" in "DatabricksQueryToolSchema" shadows an attribute in parent "BaseModel"
  warnings.warn(
/opt/homebrew/lib/python3.10/site-packages/pydantic/_internal/_generate_schema.py:502: UserWarning: <built-in function callable> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(
/opt/homebrew/lib/python3.10/site-packa

## Creating Agents

In [5]:
# Agent 1: Venue Coordinator
venue_coordinator = Agent(
    role="Venue Coordinator",
    goal="Identify and book an appropriate venue "
    "based on event requirements",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "With a keen sense of space and "
        "understanding of event logistics, "
        "you excel at finding and securing "
        "the perfect venue that fits the event's theme, "
        "size, and budget constraints."
    )
)

In [6]:
 # Agent 2: Logistics Manager
logistics_manager = Agent(
    role='Logistics Manager',
    goal=(
        "Manage all logistics for the event "
        "including catering and equipmen"
    ),
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Organized and detail-oriented, "
        "you ensure that every logistical aspect of the event "
        "from catering to equipment setup "
        "is flawlessly executed to create a seamless experience."
    )
)

In [7]:
# Agent 3: Marketing and Communications Agent
marketing_communications_agent = Agent(
    role="Marketing and Communications Agent",
    goal="Effectively market the event and "
         "communicate with participants",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Creative and communicative, "
        "you craft compelling messages and "
        "engage with potential attendees "
        "to maximize event exposure and participation."
    )
)

## Creating Venue Pydantic Object

- Create a class `VenueDetails` using [Pydantic BaseModel](https://docs.pydantic.dev/latest/api/base_model/).
- Agents will populate this object with information about different venues by creating different instances of it.

In [8]:
from pydantic import BaseModel
# Define a Pydantic model for venue details 
# (demonstrating Output as Pydantic)
class VenueDetails(BaseModel):
    name: str
    address: str
    capacity: int
    booking_status: str

## Creating Tasks
- By using `output_json`, you can specify the structure of the output you want.
- By using `output_file`, you can get your output in a file.
- By setting `human_input=True`, the task will ask for human feedback (whether you like the results or not) before finalising it.

In [9]:
venue_task = Task(
    description="Find a venue in {event_city} "
                "that meets criteria for {event_topic}.",
    expected_output="All the details of a specifically chosen"
                    "venue you found to accommodate the event.",
    human_input=True,
    output_json=VenueDetails,
    output_file="venue_details.json",  
      # Outputs the venue details as a JSON file
    agent=venue_coordinator
)

- By setting `async_execution=True`, it means the task can run in parallel with the tasks which come after it.

In [13]:
logistics_task = Task(
    description="Coordinate catering and "
                 "equipment for an event "
                 "with {expected_participants} participants "
                 "on {tentative_date}.",
    expected_output="Confirmation of all logistics arrangements "
                    "including catering and equipment setup.",
    human_input=True,
    # async_execution=True,
    agent=logistics_manager
)

In [14]:
marketing_task = Task(
    description="Promote the {event_topic} "
                "aiming to engage at least"
                "{expected_participants} potential attendees.",
    expected_output="Report on marketing activities "
                    "and attendee engagement formatted as markdown.",
    # async_execution=True,
    output_file="marketing_report.md",  # Outputs the report as a text file
    agent=marketing_communications_agent
)

## Creating the Crew

**Note**: Since you set `async_execution=True` for `logistics_task` and `marketing_task` tasks, now the order for them does not matter in the `tasks` list.

In [15]:
# Define the crew with agents and tasks
event_management_crew = Crew(
    agents=[venue_coordinator, 
            logistics_manager, 
            marketing_communications_agent],
    
    tasks=[venue_task, 
           logistics_task, 
           marketing_task],
    
    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [16]:
event_details = {
    'event_topic': "Tech Innovation Conference",
    'event_description': "A gathering of tech innovators "
                         "and industry leaders "
                         "to explore future technologies.",
    'event_city': "San Francisco",
    'tentative_date': "2024-09-15",
    'expected_participants': 500,
    'budget': 20000,
    'venue_type': "Conference Hall"
}

**Note**: 
- Since you set `human_input=True` for some tasks, the execution will ask for your input before it finishes running.
- When it asks for feedback, use your mouse pointer to first click in the text box before typing anything.

In [17]:
result = event_management_crew.kickoff(inputs=event_details)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 9e6c6022-d627-47d1-999e-c0a9f2b121cf                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: fdc562bb-7d15-41c5-bba0-e3edd2091bdb
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: fdc562bb-7d15-41c5-bba0-e3edd2091bdb
       Status: Executing Task...
    └── 🤖 Agent: Venue Coordinator
            Status: In Progress

# Agent: Venue Coordinator
## Task: Find a venue in San Francisco that meets criteria for Tech Innovation Conference.


🚀 Crew: crew
└── 📋 Task: fdc562bb-7d15-41c5-bba0-e3edd2091bdb
       Status: Executing Task...
    └── 🤖 Agent: Venue Coordinator
            Status: In Progress
        └── 🧠 Thinking...

...



# Agent: Marketing and Communications Agent
## Final Answer: 
## Marketing Activities and Attendee Engagement Report

### Tech Innovation Conference Insights

- **Venue:** The conference will be held at The Pearl, located at 601 19th St, San Francisco, CA 94107, with a capacity of 1400 attendees.
- **Logistics:** All arrangements including catering and equipment setup have been confirmed. Catering services can be sourced from companies like Above & Beyond Catering, Salt & Honey Catering & Events, and San Francisco Catering Company. Event equipment rental options include Stuart Party Rentals, Abbey Party Rents, and more.
- Based on research on past tech innovation conferences, interactive content like live Q&As and polls have been proven effective in engaging attendees.
- Strategies such as leveraging interactive technology, personalizing the experience, fostering networking opportunities, and gamification can enhance attendee engagement.
- Artificial Intelligence (AI) integration in 

🚀 Crew: crew
├── 📋 Task: fdc562bb-7d15-41c5-bba0-e3edd2091bdb
│      Assigned to: Venue Coordinator
│      Status: ✅ Completed
│   └── 🤖 Agent: Venue Coordinator
│           Status: ✅ Completed
│       ├── 🔧 Used Search the internet with Serper (1)
│       └── 🔧 Used Read website content (1)
├── 📋 Task: 050c5f70-68c4-4ea6-95c1-1824e039d2e7
│      Assigned to: Logistics Manager
│      Status: ✅ Completed
│   └── 🤖 Agent: Logistics Manager
│           Status: ✅ Completed
│       └── 🔧 Used Search the internet with Serper (3)
└── 📋 Task: 6b785ca6-ffd5-4556-adde-159aabc259eb
       Status: Executing Task...
    └── 🤖 Agent: Marketing and Communications Agent
            Status: ✅ Completed
        ├── 🔧 Failed Read website content (2)
        └── 🔧 Used Search the internet with Serper (7)

🚀 Crew: crew
├── 📋 Task: fdc562bb-7d15-41c5-bba0-e3edd2091bdb
│      Assigned to: Venue Coordinator
│      Status: ✅ Completed
│   └── 🤖 Agent: Venue Coordinator
│           Status: ✅ Completed
│       ├── 🔧 Used Search the internet with Serper (1)
│       └── 🔧 Used Read website content (1)
├── 📋 Task: 050c5f70-68c4-4ea6-95c1-1824e039d2e7
│      Assigned to: Logistics Manager
│      Status: ✅ Completed
│   └── 🤖 Agent: Logistics Manager
│           Status: ✅ Completed
│       └── 🔧 Used Search the internet with Serper (3)
└── 📋 Task: 6b785ca6-ffd5-4556-adde-159aabc259eb
       Assigned to: Marketing and Communications Agent
       Status: ✅ Completed
    └── 🤖 Agent: Marketing and Communications Agent
            Status: ✅ Completed
        ├── 🔧 Failed Read website content (2)
        └── 🔧 Used Search the internet with Serper (7)

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 6b785ca6-ffd5-4556-adde-159aabc259eb                                                                     │
│  Agent: Marketing and Communications Agent                                                                      │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 9e6c6022-d627-47d1-999e-c0a9f2b121cf                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

- Display the generated `venue_details.json` file.

In [18]:
import json
from pprint import pprint

with open('venue_details.json') as f:
   data = json.load(f)

pprint(data)

{'address': '601 19th St, San Francisco, CA 94107',
 'booking_status': 'Available for booking',
 'capacity': 1400,
 'name': 'The Pearl'}


- Display the generated `marketing_report.md` file.

**Note**: After `kickoff` execution has successfully ran, wait an extra 45 seconds for the `marketing_report.md` file to be generated. If you try to run the code below before the file has been generated, your output would look like:

```
marketing_report.md
```

If you see this output, wait some more and than try again.

In [19]:
from IPython.display import Markdown
Markdown("marketing_report.md")

## Marketing Activities and Attendee Engagement Report

### Tech Innovation Conference Insights

- **Venue:** The conference will be held at The Pearl, located at 601 19th St, San Francisco, CA 94107, with a capacity of 1400 attendees.
- **Logistics:** All arrangements including catering and equipment setup have been confirmed. Catering services can be sourced from companies like Above & Beyond Catering, Salt & Honey Catering & Events, and San Francisco Catering Company. Event equipment rental options include Stuart Party Rentals, Abbey Party Rents, and more.
- Based on research on past tech innovation conferences, interactive content like live Q&As and polls have been proven effective in engaging attendees.
- Strategies such as leveraging interactive technology, personalizing the experience, fostering networking opportunities, and gamification can enhance attendee engagement.
- Artificial Intelligence (AI) integration in event management processes led to a significant increase in participant engagement and networking activities.
- Post-event surveys are valuable for gathering feedback on attendee experience with engagement tools.

By implementing innovative and interactive engagement strategies, the Tech Innovation Conference can create a valuable and engaging experience for attendees, maximizing success and ROI.